# Placa extrema para 4E sin atiesadores

## Introducción
A continuación se muestra la revisión de la placa extrema y tornillos para trabes a momento


<img src="4E.PNG" alt="Drawing" style="width: 300px;"/>

## Importación de bilbiotecas

In [1]:
import math
import csv

## Elementos mecánicos

In [2]:
mu=25*100000
vu=15*1000

## Placa extrema

In [3]:
pe = dict()
pe["Base"] = 23
pe["fy"] = 2530
pe["fu"] = 4100
#Datos de tornillo ASTM A490
pe["de"]=4.5 
pe["pfo"]=4.5 
pe["pfi"]=4.5
pe["ft"]=7900
pe["tp"]=2
pe["db"]=2
pe["fv"]=4200

## Perfil

In [34]:
def importarPerfil(perfil):
    perfil = perfil.upper()
    csv_path = "../aisc_database.csv"
    with open(csv_path, "r") as f_obj:
        reader = csv.reader(f_obj)
        for row in reader:
            if row[3] == perfil:
                section = dict()
                section["type"] = row[2]
                section["area"] = float(row[10]) * 2.54 ** 2
                section["rx"] = float(row[28]) * 2.54
                section["ry"] = float(row[34]) * 2.54
                section["zx"] = float(row[26]) * 2.54 ** 3
                section["zy"] = float(row[32]) * 2.54 ** 3
                section["sx"] = float(row[27]) * 2.54 ** 3
                section["sy"] = float(row[33]) * 2.54 ** 3
                section["ix"] = float(row[25]) * 2.54 ** 4
                section["iy"] = float(row[31]) * 2.54 ** 4
                section["cw"] = float(row[84]) * 2.54 ** 6
                section["j"] = float(row[54]) * 2.54 ** 4
                section["h0"] = float(row[135]) * 2.54 ** 4
                section["rts"] = float(row[134]) * 2.54
                section["d"] = float(row[11]) * 2.54
                section["bf"] = float(row[14]) * 2.54
                section["tw"] = float(row[12]) * 2.54
                section["tf"] = float(row[15]) * 2.54

                return section
sectionName="W21X55"
section=importarPerfil(sectionName)
section["g"]=13.97
ColumnName="W14X109"
Column=importarPerfil(ColumnName)
Column["ts"]=1.5 #Atiesador en columna
Column["fy"]=3515 #Resistensia del acero
#section

In [35]:
#Datos de placa y tornillos
bp=pe["Base"]
g=section["g"]
pfi=pe["pfi"]
pfo=pe["pfo"]
de=pe["de"]
fy=pe["fy"]
fu=pe["fu"]
ft=pe["ft"]
fv=pe["fv"]
tp=pe["tp"]
db=pe["db"]
#Datos de perfil
d=section["d"]
bf=section["bf"]
tf=section["tf"]
#Datos de columna
dc=Column["d"]
bfc=Column["bf"]
tfc=Column["tf"]
ts=Column["ts"]
fyc=Column["fy"]

h0=d+pfo-tf/2
h1=d-pfi-tf/2

print("h0 = %.2f"%(h0), " cm")
print("h1 = %.2f"%(h1), " cm")

h0 = 56.67  cm
h1 = 47.67  cm


## Determinación del diámetro del ancla

$d=\sqrt{\frac{2Mu}{\pi \phi F_t (h_0+h1)}}$

In [6]:
dt=math.sqrt(2*mu/(math.pi*0.75*ft*(h0+h1)))
print("dt_min = %.2f"%(dt), " cm")
print("db = %.2f"%(db), " cm")
print("rel = %.2f"%(dt/db))

dt_min = 1.60  cm
db = 2.00  cm
rel = 0.80


## Espesor de placa base

$s=\max\left(\frac{1}{2}\sqrt{b_f g},pfi\right)$

In [7]:
s=max(0.5*math.sqrt(bf*g),pfi)
print("s = %.2f"%(s), " cm")

s = 8.54  cm


$Y=\frac{b_p}{2}\left[h1\left(\frac{1}{pfi}+\frac{1}{s}\right)+h0\left(\frac{1}{pfo}\right)-\frac{1}{2}\right]+\frac{2}{g}\left[h1(pfi+s)\right]$

In [8]:
Y=bp/2*(h1*(1/pfi+1/s)+h0*(1/pfo)-0.5)+2/g*(h1*(pfi+s))
print("Y = %.2f"%(s), " cm")

Y = 8.54  cm


$t=\sqrt{\frac{Mu}{\phi FyY}}$

In [9]:
t=math.sqrt(mu/(0.9*fy*Y))
print("tmin = %.2f"%(t), " cm")
print("tp = %.2f"%(tp), " cm")
print("rel = %.2f"%(t/tp))

tmin = 1.63  cm
tp = 2.00  cm
rel = 0.81


## Revisión por cortante
### Fuerza en el patín de la viga

$F_{fu}=\frac{M_u}{db-tf}$

In [10]:
Ffu=mu/(d-tf)
print("ffu = %.2f"%(Ffu), " kg")

ffu = 48537.92  kg


Revisión de cortante debido al momento

$R_n=0.9(0.6F_y)b_pt_p$

In [11]:
Rn_flex=0.9*0.9*fy*bp*tp
print("Rn = %.2f"%(Rn_flex), " kg")
print("rel = %.2f"%(Ffu/Rn_flex))

Rn = 94267.80  kg
rel = 0.51


Revisión de cortante por ruptura

$A_n=\left[b_p-2(db+2.54(1/8))\right]t_p$

In [12]:
An=(bp-2*(db+2.54/8))*tp
print("An = %.2f cm^2"%(An))

An = 36.73 cm^2


Capacidad a cortante

$R_n=0.75(0.6 F_u)A_n$

In [13]:
Rn_db=0.75*0.6*fu*An
print("Rn = %.2f kg"%(Rn_db))
print("rel = %.2f"%((Ffu/2)/Rn_db))

Rn = 67766.85 kg
rel = 0.36


### Revisión de fuerzas en tornillo

REvisión de cortante

$R_n=\phi n_bF_v(A_b)$

In [14]:
Rn_bolt_shear=0.75*4*fv*(math.pi*db**2/4)
print("Rn = %.2f kg"%(Rn_bolt_shear))
print("Vu = %.2f kg"%(vu))
print("rel = %.2f"%(vu/Rn_bolt_shear))

Rn = 39584.07 kg
Vu = 15000.00 kg
rel = 0.38


### Revisión de tensión y compresión del tornillo

Tornillos a compresión

$f_{bearing}=1088.62d_bt_pF_u$

In [15]:
Rn_bearing=1088.62*db*tp*fu
print("Rn = %.2f kg/tornillo"%(Rn_bearing))

Rn = 17853368.00 kg/tornillo


Tornillos a tensión

$L_c=(pfi+tf+pfo)-(db+2.54(1/16))$

In [16]:
Lc=(pfi+tf+pfo)-(db+2.54/16)
print("Lc = %.2f cm"%(Lc))

Lc = 8.17 cm


$R_n=544.31L_ct_pF_u$

In [17]:
Rn_tearing=5441.31+Lc*tp*fu
print("Rn = %.2f kg/tornillo"%(Rn_tearing))

Rn = 72411.78 kg/tornillo


In [18]:
Rn_bolt=min(Rn_bearing,Rn_tearing)*4*0.75
print("Rn = %.2f kg"%(Rn_bolt))
print("Vu = %.2f kg"%(vu))
print("rel = %.2f"%(vu/Rn_bolt))

Rn = 217235.33 kg
Vu = 15000.00 kg
rel = 0.07


## Diseño de atiesdaores

### Revisión de atiesdaor por flexión en la columna

$s=\frac{1}{2}\sqrt{b_{fc}g}$

In [19]:
s=0.5*math.sqrt(bfc*g)
print("s = %.2f cm"%(s))

s = 11.38 cm


$c=pfo+tfb+pfi$

In [20]:
c=pfo+t+pfi
print("c = %.2f cm"%(c))

c = 10.63 cm



${Y_c} = \frac{{{b_{fc}}}}{2}\left[ {h1\left( {\frac{1}{s}} \right) + h0\left( {\frac{1}{s}} \right)} \right] + \frac{2}{g}\left[ {h1\left( {s + \frac{{3c}}{4}} \right) + h0\left( {s + \frac{c}{4}} \right) + \frac{{{c^2}}}{2}} \right] + \frac{g}{2}$

In [21]:
Yc=bfc/2*(h1*(1/s)+h0*(1/s)+2/g*(h1*(s+3*c/4)+h0*(s+c/4)+c**2/2)+g/2)
print("Yc = %.2f"%(Yc), " cm")

Yc = 5009.89  cm


${t_{fc}} = \sqrt {\frac{{{M_{u}}}}{{\phi {F_y}{Y_c}}}} $

In [36]:
tfc_min=math.sqrt(mu/(0.9*fyc*Yc))
print("tfc_min = %.2f"%(tfc_min), " cm")
print("tfc = %.2f"%(tfc), " cm")
req_stiffner=True if tfc_min>tfc else False
if req_stiffner:
    print("Requiere atiesdaores")
else:
    print("No requiere atiesadores")

tfc_min = 0.40  cm
tfc = 2.18  cm
No requiere atiesadores


$pso=psi=\frac{c-ts}{2}$

${Y_{cs}} = \frac{{{b_f}}}{2}\left[ {h1\left( {\frac{1}{s} + \frac{1}{{psi}}} \right) + h0\left( {\frac{1}{s} + \frac{1}{{pso}}} \right)} \right] + \frac{2}{g}\left[ {h1\left( {s + psi} \right) + h0\left( {s + pso} \right)} \right]$

In [38]:
pso=(c-ts)/2
psi=pso
Ycs=bf/2*(h1*(1/s+1/psi)+h0*(1/s+1/pso))+2/g*(h1*(s+psi)+h0*(s+pso))
tfcs_min=math.sqrt(mu/(0.9*fyc*Ycs))
print("pso = %.2f"%(pso), " cm")
print("Ycs = %.2f"%(Ycs), " cm")
print("tfc_min = %.2f"%(tfcs_min), " cm")
req_stiffner_dim=True if tfcs_min>tfc else False
if req_stiffner:
    print("Aumentar espesor de atiesador")
else:
    print("Espesor adecuado")

pso = 4.56  cm
Ycs = 572.53  cm
tfc_min = 1.17  cm
Espesor adecuado
